In [1]:
import cv2
import numpy as np
from PIL import Image
import glob
import sys
import mysql.connector
from skimage import io
import os
import PIL.Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import color
import keras
import shutil
from sklearn.metrics import classification_report
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import base64
from io import BytesIO

/home/anshul/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [3]:
def Neural():
    db=mysql.connector.connect(host='localhost',database='mysql',user='root',password='Anshul@13')
    cursor = db.cursor()
    detector=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    eye=cv2.CascadeClassifier('haarcascade_eye.xml')
    mouth=cv2.CascadeClassifier('haarcascade_mouth.xml')
    car=cv2.CascadeClassifier('haarcascade_car.xml')
    v=input("Want to work on previous one: ")
    if v=='n':
        sql="select id from img"
        cursor=db.cursor()
        cursor.execute(sql)
        data=cursor.fetchall()
        check=len(data)
        Type=check
        if(check!=0):
            Type=data[len(data)-1][0]
        print (Type)
        '''
        if(os.path.isdir("C:/Users/anshul/jupyter/capture")):
            shutil.rmtree('C:/Users/anshul/jupyter/capture')
            os.mkdir('C:/Users/anshul/jupyter/'+str('capture'))
            os.mkdir('C:/Users/anshul/jupyter/capture/'+str('mouth'))
            os.mkdir('C:/Users/anshul/jupyter/capture/'+str('eye'))

        else:
            os.mkdir('C:/Users/anshul/jupyter/'+str('capture'))
            os.mkdir('C:/Users/anshul/jupyter/capture/'+str('mouth'))
            os.mkdir('C:/Users/anshul/jupyter/capture/'+str('eye'))
        '''    
        l=[]
        entries=int(input("How many Entries do you want enter?\n"))
        for i in range(entries):
            a=0
            first_name=input("\nEnter your first name: ")
            last_name=input("Enter your last name: ")
            email=input("Enter your email id: ")
            branch=input("Enter your branch: ")
            sql="insert into student values("+str(int(Type)+i+1)+",'"+str(first_name)+"','"+str(last_name)+"','"+str(email)+"','"+str(branch)+"')"
            cursor.execute(sql)
            '''
            LABEL=input('\nEnter the label: ')
            print('\n')
            l.append(LABEL)
            '''
            cap = cv2.VideoCapture(0)
            a=0
            
            while True:
                flag, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = detector.detectMultiScale(gray, 1.3, 5)
                for (x,y,w,h) in faces:
                    face1=cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
                    roi_gray=gray[y:y+h, x:x+w]
                    roi_color=frame[y:y+h, x:x+w]
                    cv2.imwrite('/home/anshul/computer_vision/Face detection/test/'+str(int(Type)+i+1)+'.'+str(a)+'.png', roi_color)
                    #with open('/home/anshul/computer_vision/Face detection/test/'+str(a)+'.png', "rb") as image_file:
                    #if (os.path.isfile('/home/anshul/computer_vision/Face detection/test/'+str(int(Type)+i+1)+'.'+str(a)+'.png')):    
                    a_i_p=glob.glob(os.path.join('/home/anshul/computer_vision/Face detection/test/'+str(int(Type)+i+1)+'.'+str(a)+'.png'))
                    for i_p in a_i_p:
                        img=Image.open(i_p)
                        img=img.resize((100,100), Image.ANTIALIAS)
                        img.save(i_p)
                        image_file = open('/home/anshul/computer_vision/Face detection/test/'+str(int(Type)+i+1)+'.'+str(a)+'.png', "rb")   
                        encoded_string = base64.b64encode(image_file.read())
                        sql = 'INSERT INTO img(images,id) VALUES(%s,%s)'    
                        args = (encoded_string,(int(Type)+i+1), )
                        cursor=db.cursor()
                        cursor.execute(sql,args)
                        db.commit()
                        os.remove('/home/anshul/computer_vision/Face detection/test/'+str(int(Type)+i+1)+'.'+str(a)+'.png')
                        a=a+1

                sys.stdout.write('\r')
                per=int((a/2000)*100)
                sys.stdout.write('Scanning '+str(per)+' % completed')
                sys.stdout.flush()

                cv2.imshow('video', frame)
                key=cv2.waitKey(1)    
                if (cv2.waitKey(1) & 0xFF == ord('q')) or (per==100):
                    break
            cap.release()
            cv2.destroyAllWindows()  
        
        '''
        sql1='select images from img'
        cursor=db.cursor()
        cursor.execute(sql1)
        data=cursor.fetchall()
        '''
        sql2='select id from img'
        cursor=db.cursor()
        cursor.execute(sql2)
        ide=cursor.fetchall()
        for j in range(0,len(np.unique(ide))):
            sql3="select images from img where id="+str(j+1)
            cursor=db.cursor()
            cursor.execute(sql3)
            data=cursor.fetchall()
            for i in range(0,len(data)):
                data1=base64.b64decode(data[i][0])
                file_like=BytesIO(data1)
                img=PIL.Image.open(file_like)
                img.save('/home/anshul/computer_vision/Face detection/test/'+str(j+1)+'.'+str(i)+'.png')

        db.commit()
        db.close()
                
    def get_class(img_path):
        path_list=img_path.split(os.sep)
        Type=path_list[6].split(".")
        return (int(Type[0])-1)
        

    
    
    def take_data(path):
        imgs=[]
        labels=[]
        i=0
        all_img_path=glob.glob(os.path.join(path))
        np.random.shuffle(all_img_path)
        for img_path in all_img_path:
            img=cv2.imread(img_path,0)
            imgs.append(img)
            label = get_class(img_path)
            
            labels.append(label)
            
            sys.stdout.write('\r')
            sys.stdout.write('Image Processing: '+str(i))
            i=i+1
     
        X = np.array(imgs)
        newX=[]
    
        for k in range(0,len(X)):
            newX.append(X[k])
        newX=np.array(newX)

        Y = np.array(labels)
        return (newX,Y)
        

    '''
    def imresize(path):
        a_i_p=glob.glob(os.path.join(path))
        i=0
        print ('\n')
        
        for i_p in a_i_p:
            img=Image.open(i_p)
            img=img.resize((100,100), Image.ANTIALIAS)
            img.save(i_p)
            sys.stdout.write('\r')
            sys.stdout.write('Image Resize: '+str(i))
            i=i+1
            
        print("Resizing Complete")
    '''
    #imresize("/home/anshul/computer_vision/Face detection/capture/*.png")
    #imresize("C:/Users/anshul/jupyter/capture/eye/*.png")
    #imresize("C:/Users/anshul/jupyter/capture/mouth/*.png")
    train_X,train_Y=take_data("/home/anshul/computer_vision/Face detection/test/*.png")
    #train_eye_X,train_eye_Y=take_data("C:/Users/anshul/jupyter/capture/eye/*.png",'train')
    #train_mouth_X,train_mouth_Y=take_data("C:/Users/anshul/jupyter/capture/mouth/*.png",'train')
    print('\nTrain Shape: ',train_X.shape,train_Y.shape)
    classes=np.unique(train_Y)
    nClasses=len(classes)
    print("Classes are: ",classes)
    print("Total no. of classes: ",nClasses)
    plt.figure(figsize=[5,5])

    plt.subplot(121)
    plt.imshow(train_X[0,:,:], cmap='gray')

    plt.show()
    train_X=np.reshape(train_X,[-1,100,100,1])
    train_X=train_X/255
    train_Y_one_hot=to_categorical(train_Y)
    train_data,val_data,train_label,val_label=train_test_split(train_X,train_Y_one_hot,test_size=0.2, random_state=13)
    batchsize=100
    epochs=2
    model=Sequential()
    shape=np.array([32,64,128])
    for i in range(0,len(shape)):
        if i==0:
            model.add(Conv2D(shape[i], kernel_size=(3,3), activation='linear', input_shape=(100,100,1), padding='same'))
        else:
            model.add(Conv2D(shape[i], kernel_size=(3,3), activation='linear', padding='same'))
        model.add(Activation('relu'))
        model.add(Dropout(0.3))
        model.add(MaxPooling2D(2,2 ,padding='same'))
    model.add(Flatten())
    model.add(Dense(128, activation='linear'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    model.summary()
    model_train = model.fit(train_data, train_label, batch_size=batchsize,epochs=epochs,
                        verbose=1,validation_data=(val_data, val_label))
    
    shutil.rmtree('/home/anshul/computer_vision/Face detection/test')
    os.mkdir('/home/anshul/computer_vision/Face detection/test')
            
    return model